# Using Arcee Maestro 7B Preview on SageMaker through the Hugging Face hub

[Arcee-Maestro-7B-Preview](https://huggingface.co/arcee-ai/Arcee-Maestro-7B-Preview) is Arcee's first reasoning model trained with reinforment learning. It is based on the Qwen2.5-7B DeepSeek-R1 distillation DeepSeek-R1-Distill-Qwen-7B with further GPRO training. Though this is just a preview of our upcoming work, it already shows promising improvements to mathematical and coding abilities across a range of tasks.

## 1. Import dependencies

In [ ]:
%%sh
pip install -qU sagemaker huggingface_hub

In [ ]:
import datetime
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.djl_inference.model import DJLModel
from sagemaker_streaming import print_event_stream

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

In this example, we're deploying Arcee Maestro 7B Preview on a SageMaker real-time endpoint hosted on a GPU instance. If you need general information on real-time inference with Amazon SageMaker, please refer to the SageMaker [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

The endpoint runs a DJLServing [Deep Learning Container](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-models-frameworks-djl-serving.html), powered by vLLM. vLLM enables high-performance text generation for the most popular open-source language models. 

The endpoint configuration focuses on cost efficiency. It uses a [g6e.2xlarge](https://aws.amazon.com/ec2/instance-types/g6e/) instance. This instance has a single NVDIA L40S GPU, with 48GB of GPU RAM. Arcee Maestro 7B Preview has 7 billion 16-bit parameters, which can easily fit without the need for quantization.

#### OpenAI compatibility

The [OpenAI Messages API](https://huggingface.co/docs/text-generation-inference/messages_api) is available. This alllows you to invoke the endpoint in the same way you would invoke an OpenAI model. Likewise, the output format is identical to the OpenAI models.

### A. Define the endpoint configuration

In [ ]:
model_id = "arcee-ai/Arcee-Maestro-7B-Preview"
endpoint_name_prefix = "Arcee-Maestro-7B-Preview"

In [ ]:
# g6e endpoint

real_time_inference_instance_type = "ml.g6e.2xlarge"

### B. Create the endpoint

In [ ]:
model = DJLModel(model_id=model_id, role=role, env=None)

# create a unique endpoint name
timestamp = "{:%Y-%m-%d-%H-%M-%S}".format(datetime.datetime.now())
endpoint_name = f"{endpoint_name_prefix}-{timestamp}"
print(f"Deploying endpoint {endpoint_name}")

# deploy the model
response = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name,
    model_data_download_timeout=900,
    container_startup_health_check_timeout=900,
)

Once the endpoint is in service, you will be able to perform real-time inference.

### C. Perform streaming inference

In [ ]:
model_sample_input = {
    "messages": [
        {"role": "system", "content": "You are a friendly and helpful AI financial assistant."},
        {
            "role": "user",
            "content": """Explain the impact of rising U.S. interest rates on emerging market bonds. Give an example from recent history.""",
        },
    ],
    "max_tokens": 8192,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

In [ ]:
# https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu-aarch64.cpp

model_sample_input = {
    "messages": [
        {"role": "system", "content": "You are a friendly and helpful AI coding assistant."},
        {
            "role": "user",
            "content": """In the code below, analyse how Neon instructions are used for vectorization. Write a clear and compact summary.
            
static void ggml_gemv_q4_0_4x4_q8_0(int n, float * GGML_RESTRICT s, size_t bs, const void * GGML_RESTRICT vx, const void * GGML_RESTRICT vy, int nr, int nc) {
    const int qk = QK8_0;
    const int nb = n / qk;
    const int ncols_interleaved = 4;
    const int blocklen = 4;

    assert (n % qk == 0);
    assert (nc % ncols_interleaved == 0);

    UNUSED(s);
    UNUSED(bs);
    UNUSED(vx);
    UNUSED(vy);
    UNUSED(nr);
    UNUSED(nc);
    UNUSED(nb);
    UNUSED(ncols_interleaved);
    UNUSED(blocklen);

#if ! ((defined(_MSC_VER)) && ! defined(__clang__)) && defined(__aarch64__) && defined(__ARM_NEON) && defined(__ARM_FEATURE_DOTPROD)
    if (ggml_cpu_has_neon() && ggml_cpu_has_dotprod()) {
        const block_q4_0x4 * b_ptr = (const block_q4_0x4 *) vx;

        for (int c = 0; c < nc; c += ncols_interleaved) {
            const block_q8_0 * a_ptr = (const block_q8_0 *) vy;
            float32x4_t acc = vdupq_n_f32(0);
            for (int b = 0; b < nb; b++) {
                int8x16_t b0 = vld1q_s8((const int8_t *) b_ptr->qs);
                int8x16_t b1 = vld1q_s8((const int8_t *) b_ptr->qs + 16);
                int8x16_t b2 = vld1q_s8((const int8_t *) b_ptr->qs + 32);
                int8x16_t b3 = vld1q_s8((const int8_t *) b_ptr->qs + 48);
                float16x4_t bd = vld1_f16((const __fp16 *) b_ptr->d);

                int8x16_t a0 = vld1q_s8(a_ptr->qs);
                int8x16_t a1 = vld1q_s8(a_ptr->qs + qk/2);
                float16x4_t ad = vld1_dup_f16((const __fp16 *) &a_ptr->d);

                int32x4_t ret = vdupq_n_s32(0);

                ret = vdotq_laneq_s32(ret, b0 << 4, a0, 0);
                ret = vdotq_laneq_s32(ret, b1 << 4, a0, 1);
                ret = vdotq_laneq_s32(ret, b2 << 4, a0, 2);
                ret = vdotq_laneq_s32(ret, b3 << 4, a0, 3);

                ret = vdotq_laneq_s32(ret, b0 & 0xf0U, a1, 0);
                ret = vdotq_laneq_s32(ret, b1 & 0xf0U, a1, 1);
                ret = vdotq_laneq_s32(ret, b2 & 0xf0U, a1, 2);
                ret = vdotq_laneq_s32(ret, b3 & 0xf0U, a1, 3);

                acc = vfmaq_f32(acc, vcvtq_n_f32_s32(ret, 4),
                                vmulq_f32(vcvt_f32_f16(ad), vcvt_f32_f16(bd)));
                a_ptr++;
                b_ptr++;
            }
            vst1q_f32(s, acc);
            s += ncols_interleaved;
        }
        return;
    }
#endif // #if ! ((defined(_MSC_VER)) && ! defined(__clang__)) && defined(__aarch64__) && defined(__ARM_NEON) && defined(__ARM_FEATURE_DOTPROD)
    float sumf[4];
    int sumi;

    const block_q8_0 * a_ptr = (const block_q8_0 *) vy;
    for (int x = 0; x < nc / ncols_interleaved; x++) {
        const block_q4_0x4 * b_ptr = (const block_q4_0x4 *) vx + (x * nb);

        for (int j = 0; j < ncols_interleaved; j++) sumf[j] = 0.0;
        for (int l = 0; l < nb; l++) {
            for (int k = 0; k < (qk / (2 * blocklen)); k++) {
                for (int j = 0; j < ncols_interleaved; j++) {
                    sumi = 0;
                    for (int i = 0; i < blocklen; ++i) {
                        const int v0 = (int8_t) (b_ptr[l].qs[k * ncols_interleaved * blocklen + j * blocklen + i] << 4);
                        const int v1 = (int8_t) (b_ptr[l].qs[k * ncols_interleaved * blocklen + j * blocklen + i] & 0xF0);
                        sumi += ((v0 * a_ptr[l].qs[k * blocklen + i]) + (v1 * a_ptr[l].qs[k * blocklen + i + qk / 2])) >> 4;
                    }
                    sumf[j] += sumi * GGML_FP16_TO_FP32(b_ptr[l].d[j]) * GGML_FP16_TO_FP32(a_ptr[l].d);
                }
            }
        }
        for (int j = 0; j < ncols_interleaved; j++) s[x * ncols_interleaved + j] = sumf[j];
    }
}

    """,
        },
    ],
    "max_tokens": 8192,
    "stream": True
}

response = runtime_sm_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(model_sample_input),
    ContentType='application/json'
)

print_event_stream(response['Body'])

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

## 4. Clean-up

Please don't forget to run the cells below to delete all resources and avoid unecessary charges.

### A. Delete the endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()

Thank you for trying out Arcee Maestro 7B on SageMaker. We have only scratched the surface of what you can do with this model.

We'd be happy to hear from you, learn more about your use case, and help you build your next AI-driven solution. Please reach out to julien@arcee.ai.